# **Consommation et revenu disponible brut des ménages français depuis 1949**

Il est temps de mettre en pratique ce qui a été vu sur les séries temporelles via un petit projet guidé d'analyse de séries temporelles.

L'objectif est de s'intéresser à la relation entre les dépenses de consommation et le revenu disponible des ménages en France. Pour ce faire, on distinguera trois parties :
1. [l'acquisition des données](#donnees)
2. [la modélisation univariée à partir d'une spécification ARIMA](#arima)
3. [l'existence d'une relation de long terme et d'un modèle à correction d'erreur](#extra) : ça, ça risque d'être un peu plus coton parce qu'on en a pas parlé et que c'est tout simplement plus dur, mais pour les plus rapides, peut-être vous aurez le temps de vous penchez dessus et sinon je vous ferai une correction que vous sachiez à quoi ça correspond.

Vous trouverez dans le dossier data, les series suivantes nécéssaires à votre travail :
- la série de [consommation des ménages aux prix courants](https://www.insee.fr/fr/statistiques/4485021?sommaire=4485040)
- la série du [revenu disponible brut des ménages aux prix courants](https://www.insee.fr/fr/statistiques/4485029?sommaire=4485040) (souvent abrégé *RDB*)
- vous aurez également besoin des [indices de prix à la consommation](https://www.insee.fr/fr/metadonnees/source/indicateur/p1653/description) pour transformer en valeurs réelles les séries qui sont en valeurs nominales.

<a id="donnees"></a>

## **Partie 1 - Les données**

In [1]:
#vos imports
import fonctions as fct
import sys
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from scipy.stats import shapiro
import warnings ; warnings.filterwarnings(action='ignore') #pour ignorer les warnings : privilégier en général action="once" pour n'avoir qu'une fois les warnings

### **Question 1.1**

Importer les 2 séries en gérant bien les dates afin d'avoir un DatetimeIndex de frequence trimestriel.

### **Question 1.2**

Qu'appelle-t-on "valeur nominale" et "valeur réelle" ?  
En quelles valeurs sont les séries dont vous disposez ? La réponse est dans l'énoncé...  
Quelles valeurs vous paraissent plus adaptées au problème ?  
Faire la transformation nécessaire (vous disposez de la série des indices de prix à la consommation, ça peut servir)

### **Question 1.3**


Représentez graphiquement les 2 séries avec et sans transformation logarithmique.  
Que constate-t-on ?  
Laquelle vous semble-t-elle le plus adéquate ?

### **Question 1.4**

La série est-t-elle stationnaire ?  
Pour ceux que ça intéresserait (y a un peu de maths derrière par contre...), il existe des tests pour déterminer si une série est stationnaire lorsque ce n'est pas évident graphiquement. Il en existe de 2 sortes :
- les tests de stationnarité, comme le test KPSS, pour lesquels l'hypothèse nulle $H_0$ est que la série est stationnaire
- les tests de racine unitaire comme le test de Dickey-Fuller, le test augmenté de Dickey-Fuller (ADF), ou encore le test de Phillips-Perron (PP) pour lesquels l'hypothèse nulle $H_0$ est que la série a été générée par un processus présentant une racine unitaire, et donc, qu'elle n'est pas stationnaire

<a id="arima"></a>

## **Partie 2 - Modèle ARIMA**

Il est temps de modéliser ces séries par un modèle ARIMA : dans un premier temps, la série consommation puis il conviendra de répéter les opérations pour la série revenu disponible.

### **Question 2.1**

Se ramener à une série stationnaire par différenciation.

### **Question 2.2**

Proposer un ordre de grandeur maximum pour les parties *autorégressive* et *moyenne mobile*, c'est-à-dire pour les valeurs $p_{max}$ et $q_{max}$ du modèle $ARMA(p,q)$.

**Indication :** La détermination de bornes supérieures concernant l'ordre de la partie AR et de la partie MA passe par l'étude des autocorrélogrammes et des autocorrélogrammes partiels. En effet, si la série suit un modèle $ARMA(p,q)$, l'autocorrélogramme devient nul à partir du rang $q$ et l'autocorrélogramme partiel devient nul à partir du rang $p$. Dès lors, en se basant sur les autocorrélogrammes, il est possible de déterminer empiriquement les ordres maximum en regardant à partir de quel rang les autocorrélogrammes sont statistiquement nuls.

### **Question 2.3**

Déterminer, à l'aide du critère $AIC$ (vous pouvez aussi utiliser le critère $BIC$), le ou les modèles les plus pertinents.

### **Question 2.4**

Pour valider les modèles  retenus, il reste deux critères à regarder :
1. regarder la significativité des coefficients
2. examiner la "blancheur" des résidus

### **Question 2.5**

Répéter l'analyse pour la série des revenus disponibles.

<a id="extra"></a>

## **Partie 3 - Cointégration et modèle à correction d'erreur**

On s'intéresse dans cette partie au lien qu'il semble exister entre nos deux séries. En effet, graphiquement, les 2 séries ont l'air d'être liées par une relation de long terme. Plus spécifiquement, on suppose ici qu'il existe une éventuelle relation de **cointégration** de manière à mettre en place, par la suite, un modèle à **correction d'erreur**, ce qu'on ne fera pas ici.

On parle de cointégration lorsque les deux séries sont intégrées d’ordre 1 (c'est-à-dire que leur différenciation une fois : la série $x_t-x_{t-1}$ est stationnaire) et qu'il existe une relation linéaire entre les deux qui soit non intégrée (ou encore intégrée d'ordre zéro ou stationnaire, tout ça c'est à peu près pareil).

Sans forcément aller trop loin sur la partie théorique, juste 2 petites questions guidées permettent de regarder en pratique un début de ce qu'est la cointégration des séries.

### **Question 3.1**

Sous réserve de validation, la relation de long-terme est modélisée par un modèle linéaire. On va commencer par effectuer une régression linéaire de la série consommation, notée $C_t$ sur la série revenu disponible notée $R_t$ (avec ou sans transformation logarithmique selon l'étude qui a précédée) :

$$C_t = \beta_0 + \beta_1 R_t + z_t$$

Utiliser pour cette régression la libraire `statsmodel` afin afficher le `summary` de la régression puis commenter les résultats (notamment la significativité et les valeurs des coefficients)

### **Question 3.2**

Pour valider le modèle de cointégration, les résidus $z_t$ de la régression linéaire doivent être stationnaire. On peut regarder déjà graphiquement ce qu'il en est. Commenter le résultat.

Sans entrer dans le détail des tests de stationnarité évoqués précédemment, un peut utiliser par exmple la fonction `adfuller` du module `statsmodels.tsa.stattools` pour vérifier si la série est stationnaire. L'hypothèse nulle $H_0$ du test ADF est que la série admet ce qu'on appelle une racine unitaire et donc que la série n'est pas stationnaire.

**Pour aller plus loin :** si vous souhaitez pousser l'analyse, vous pouvez vous intéresser à ce qu'on appelle un modèle à correction d'erreurs et essayer de comprendre la notion de vitesse d'ajustement. Pour cela, il faut, entre autres, modéliser les résidus par un processus $ARMA$.